<a href="https://colab.research.google.com/github/Tomato-in-Japan/Subject_Fri1/blob/Homeworks/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9%E5%85%A5%E9%96%80_%E7%AC%AC10%E5%9B%9E_%E3%83%AF%E3%83%BC%E3%82%AF%E3%82%B7%E3%83%BC%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第10回 データサイエンス入門 ワークシート

次のセルに「学籍番号」、「氏名」、「クラス」、「出席番号」を入力して下さい。

In [ ]:
# 学籍番号：b2220630　氏名：加賀大智　クラス：２A　出席番号：17

# 1. 導入

## 1-1. 概要

今回は、教師あり学習の手法として「ロジスティック回帰」について学びます。
ロジスティック「回帰」という名前ですが、回帰ではなく「分類」のアルゴリズムです。つまり、今回は目的変数 $y$ が離散型変数です。

機械学習の概要については講義ビデオ「１０回目（１）」を用意していますので、確認してください。

$d$ 個の説明変数 $x_1, x_2, \cdots, x_d$ と目的変数 $p(x)$ について、以下のロジスティックモデル

\begin{align}
p(x) &= \frac{\mathrm{exp}(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_d)}{1 + \mathrm{exp}(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_d)} \\
&= \frac{\mathrm{exp}(\beta^T x)}{1 + \mathrm{exp}(\beta^T x)}
\end{align}

を考えます。ただし、

$$
\beta = (\beta_0, \beta_1, \cdots, \beta_d), \quad
x = (x_1, x_2, \cdots, x_d)
$$

です。$p(x)$ の値域は $(0, 1)$ なので，しきい値 $c \in (0, 1)$ を用いて分類を行います。

偏回帰係数 $\beta$ を推定すれば、予測値の計算が可能になります。

ロジステック回帰の数学的な背景については講義ビデオ「１０回目（２）」を用意していますので、確認してください。

本ワークシートの事前解説動画（講義ビデオ「１０回目（３）」）を用意していますので、確認してください。

必要に応じて「ウェブなどで調べる」、「修学支援室／メディアコンサルタントで質問する」などしてください。

## 1-2. 本ワークシートの使い方

基本的に、ワークシート単体で学習が可能な設計になっています。

1.   入力済みセル
2.   穴埋め用セル
3.   自由記述用セル

がありますので、手を動かしながら学習を進めて下さい。

## 1-3. ライブラリのインポート

本ワークシートで使うライブラリをインポートします。以下のセルを実行して下さい。

In [1]:
# 入力済みセル1

# データの加工・処理・分析ライブラリ
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機械学習ライブラリ
import sklearn

# 小数点第 3 位まで表示
%precision 3

'%.3f'

# 2. 知識定着

## 2-1. 準備

以下の URL で公開されている個人に関するデータ（米国の国勢調査）の分析を行います：

https://archive.ics.uci.edu/ml/datasets/adult

http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

事前に一部の前処理を行った .csv ファイルをポータルに添付しています。

adult.csv を pandas の DataFrame として読み込んで下さい。


In [3]:
# 穴埋め用セル2（模範解答は最下部）
# adult.csv を pandas の DataFrame オブジェクト df として読み込み
df = pd.read_csv('adult.csv')

まずは df の中身を確認してみましょう。以下のセルを実行して下さい。

In [4]:
# 入力済みセル3
# df を表示
df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,flg_50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,0,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,1,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,0,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,1,0,0,20,United-States,0


「32561 rows × 15 columns」とあるので、32561行、15列のデータであることが分かります。

データ項目の説明は以下の通りです。カテゴリ変数 race、sex、flg_50K は事前に数字を割り当てています：

カラム名 | 説明
--- | ---
age | 年齢
workclass | 雇用形態
fnlwgt | final weightの略。国勢調査局によって割り当てられた重みらしいが、良く分からない。
education | 学歴
education_num | 学歴を数字で表したもの。数字が大きいほど、高学歴。
marital_status | 結婚歴
occupation | 職業
relationship | 家族構成における位置
race | 人種
sex | 性別（0: Female、1: Male）
capital_gain | 投資による利益
capital_loss | 投資による損失
hours_per_week | 一週間の労働時間
native_country | 出身国
flg_50K | 収入（年収）が5万ドル以上か（0: <=50K、1: >50K）

欠損値の総数を確認します。以下のセルを実行して下さい。

In [5]:
# 入力済みセル4
# データの欠損数を出力
print('データの欠損数：', df.isnull().sum().sum())

データの欠損数： 0


## 2-2. 問題設定

【例題】目的変数を「flg_50K」、説明変数を「age、education_num、capital_gain、capital_loss」として、ロジスティック回帰分析を実行せよ。

## 2-3. モデルの構築と評価

偏回帰係数の有意性を担保する為、多重共線性の有無を確認します。説明変数間の相関係数を求めて下さい。相関係数の絶対値の値が大きい組み合わせがなければＯＫです。

In [6]:
# 穴埋め用セル5（模範解答は最下部）
# 説明変数同士の相関係数の計算
df[['age', 'education_num', 'capital_gain', 'capital_loss']].corr()

,age,education_num,capital_gain,capital_loss
age,1.000000,0.036527,0.077674,0.057775
education_num,0.036527,1.000000,0.122630,0.079923
capital_gain,0.077674,0.122630,1.000000,-0.031615
capital_loss,0.057775,0.079923,-0.031615,1.000000


In [7]:
# 入力済みセル6
# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 目的変数に「flg_50K」、説明変数に「age」、「education_num」、「capital_gain」、「capital_loss」を指定
x = df[['age', 'education_num', 'capital_gain', 'capital_loss']]
y = df['flg_50K']

# データを訓練データとテストデータに分ける
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)

# モデルの初期化と学習
model = LogisticRegression(penalty='none')
model.fit(x_train, y_train)

print('訓練データに対する正解率：', model.score(x_train, y_train))
print('テストデータに対する正解率：', model.score(x_test, y_test))
print('偏回帰係数：')
print(pd.Series(model.coef_[0], index=x.columns))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


訓練データに対する正解率： 0.8104422604422604
テストデータに対する正解率： 0.8100239543025612
偏回帰係数：
age              0.039033
education_num    0.326322
capital_gain     0.000310
capital_loss     0.000703
dtype: float64


以上で学習と評価は完了です。

LogisticRegression オブジェクトの詳細は以下のウェブページを参照して下さい：

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic#sklearn.linear_model.LogisticRegression

## 2-4. 標準化による偏回帰係数の解釈

上記のモデルでは「age」、「education_num」、「capital_gain」、「capital_loss」の単位や大きさがバラバラである為、偏回帰係数の大きさに意味がありません。

標準化した説明変数から算出される偏回帰係数を**標準偏回帰係数**と呼びます。標準偏回帰係数は各変数の重要度を表す指標であり、大小の比較に意味があります。

標準化とは、以下のようにデータを変換することである：

$$
z_i = \frac{x_i - （x の平均）}{（x の標準偏差）}
$$

標準化については「復習_データ活用基礎_講義スライド（stat_lecture_slides_10.pdf）」を参照して下さい。

上記のロジスティック回帰分析に StandardScaler クラスを利用した標準化を追加した分析スクリプトは以下の通りです。

In [8]:
# 入力済みセル7
# adult.csv を pandas の DataFrame オブジェクト df として読み込み
df = pd.read_csv('adult.csv')

# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 目的変数に「flg_50K」、説明変数に「age」、「education_num」、「capital_gain」、「capital_loss」を指定
x = df[['age', 'education_num', 'capital_gain', 'capital_loss']]
y = df['flg_50K']

# データを訓練データとテストデータに分ける
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)

# 標準化ライブラリのインポート
from sklearn.preprocessing import StandardScaler

# 標準化処理
sc = StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

# モデルの初期化と学習
model = LogisticRegression(penalty='none')
model.fit(x_train_std, y_train)

print('訓練データに対する正解率：', model.score(x_train_std, y_train))
print('テストデータに対する正解率：', model.score(x_test_std, y_test))
print('偏回帰係数：')
print(pd.Series(model.coef_[0], index=x.columns))

訓練データに対する正解率： 0.8104422604422604
テストデータに対する正解率： 0.8100239543025612
偏回帰係数：
age              0.531363
education_num    0.839657
capital_gain     2.308811
capital_loss     0.286848
dtype: float64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


「2-3. モデルの構築と評価」と比較すると、

*   正解率：変化なし
*   偏回帰係数：変化あり

であることが分かる。偏回帰係数の絶対値の大きさが説明変数に対する影響力を表している。

# 3. 知識活用（課題）

【問題】目的変数を「flg_50K」、説明変数を「age、education_num、sex、capital_gain、capital_loss、hours_per_week」として、ロジスティック回帰分析を実行せよ。ただし、説明変数を標準化し、偏回帰係数を比較すること。

※分析結果の解釈には、統計的差別に至らないよう、十分なドメイン知識が必要です。「復習_データ活用基礎_講義スライド（stat_lecture_slides_01.pdf）」を参照して下さい。

In [54]:
#ライブラリ
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline
import sklearn
%precision 3

'%.3f'

In [55]:
df = pd.read_csv('adult.csv')
print('データの欠損数：', df.isnull().sum().sum())
df[['age', 'education_num', 'sex' ,'capital_gain', 'capital_loss', 'hours_per_week']].corr()

データの欠損数： 0


,age,education_num,sex,capital_gain,capital_loss,hours_per_week
age,1.000000,0.036527,0.088832,0.077674,0.057775,0.068756
education_num,0.036527,1.000000,0.012280,0.122630,0.079923,0.148123
sex,0.088832,0.012280,1.000000,0.048480,0.045567,0.229309
capital_gain,0.077674,0.122630,0.048480,1.000000,-0.031615,0.078409
capital_loss,0.057775,0.079923,0.045567,-0.031615,1.000000,0.054256
hours_per_week,0.068756,0.148123,0.229309,0.078409,0.054256,1.000000


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
x = df[['age', 'education_num', 'sex' ,'capital_gain', 'capital_loss', 'hours_per_week']]
y = df['flg_50K']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)
print(x_train_std)

[[ 3.11  -0.414 -1.427 -0.143 -0.219 -0.448]
 [ 2.669 -0.025  0.701 -0.143 -0.219 -0.039]
 [ 1.714 -0.414 -1.427 -0.143 -0.219 -0.039]
 ...
 [-1.15  -0.025 -1.427 -0.143 -0.219 -0.039]
 [ 0.466  2.307  0.701  0.89  -0.219  0.37 ]
 [-1.004 -0.025  0.701  0.206 -0.219  0.615]]


In [57]:
model = LogisticRegression(penalty='none')
model.fit(x_train_std, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


LogisticRegression(penalty='none')

In [58]:
model = LogisticRegression(penalty='none')
model.fit(x_train_std, y_train)

print('訓練データに対する正解率：', model.score(x_train_std, y_train))
print('テストデータに対する正解率：', model.score(x_test_std, y_test))
print('偏回帰係数：')
print(pd.Series(model.coef_[0], index=x.columns))

訓練データに対する正解率： 0.822051597051597
テストデータに対する正解率： 0.823536637798661
偏回帰係数：
age               0.555492
education_num     0.837195
sex               0.571265
capital_gain      2.231961
capital_loss      0.269165
hours_per_week    0.406942
dtype: float64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


# 4. 知識応用（発展課題）

【問題】標準化関数を自作し、「3. 知識活用（課題）」に適用せよ。ただし、引数と戻り値は pandas の DataFrame とする。

In [70]:
# 自由記述用セル9
# 標準化関数


def standardization(df):
  import numpy as np
  for i in df.columns.values:
    df[i]=(df[i]-df[i].mean())/df[i].std()
  return df


In [71]:
df = pd.read_csv('adult.csv')
x=df[['age', 'education_num', 'sex' ,'capital_gain', 'capital_loss', 'hours_per_week']]
y = df['flg_50K']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)
x_1=standardization(x_train)
x_2=standardization(x_test)
print(x_1)

            age  education_num       sex  capital_gain  capital_loss  \
7872   3.109986      -0.413498 -1.427103     -0.143496     -0.218979   
31180  2.669252      -0.024874  0.700677     -0.143496     -0.218979   
25875  1.714328      -0.413498 -1.427103     -0.143496     -0.218979   
4345   1.273594      -0.413498  0.700677     -0.143496     -0.218979   
15278  0.392126      -0.024874  0.700677      0.552837     -0.218979   
...         ...            ...       ...           ...           ...   
13123  0.318670       1.529624  0.700677     -0.143496     -0.218979   
19648  0.392126      -1.579371  0.700677     -0.143496     -0.218979   
9845  -1.150443      -0.024874 -1.427103     -0.143496     -0.218979   
10799  0.465582       2.306872  0.700677      0.890379     -0.218979   
2732  -1.003532      -0.024874  0.700677      0.205746     -0.218979   

       hours_per_week  
7872        -0.448367  
31180       -0.039422  
25875       -0.039422  
4345        -0.448367  
15278        0.

In [72]:
model = LogisticRegression(penalty='none')
model.fit(x_1, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


LogisticRegression(penalty='none')

In [73]:
print('訓練データに対する正解率：', model.score(x_1, y_train))
print('テストデータに対する正解率：', model.score(x_2, y_test))
print('偏回帰係数：')
print(pd.Series(model.coef_[0], index=x.columns))

訓練データに対する正解率： 0.822051597051597
テストデータに対する正解率： 0.8236594803758983
偏回帰係数：
age               0.555509
education_num     0.837221
sex               0.571282
capital_gain      2.232030
capital_loss      0.269173
hours_per_week    0.406954
dtype: float64


In [ ]:
# 自由記述用セル10
# セルは必要に応じて追加して下さい。


# 5. 模範解答



```
# 穴埋め用セル2（模範解答は最下部）
# adult.csv を pandas の DataFrame オブジェクト df として読み込み
df = pd.read_csv('adult.csv')
```
```
# 穴埋め用セル5（模範解答は最下部）
# 説明変数同士の相関係数の計算
df[['age', 'education_num', 'capital_gain', 'capital_loss']].corr()
```

